In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/AK_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
356,5048318601,AK,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",64.030600,-145.731320,NaN
392,2725348408,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
415,5085327008,AK,"1229 Richardson Hwy, DELTA JUNCTION, AK",64.030600,-145.731320,NaN
1095,4188937110,AK,"525 SECOND STREET, SKAGWAY, AK",40.670782,-73.976908,NaN
1601,9864858410,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
1691,5628247103,AK,"6100 A Street, ANCHORAGE, AK",61.165676,-149.882046,NaN
1847,9635677110,AK,"8840 Old Seward Hwy Ste E, ANCHORAGE, AK",61.140366,-149.865092,NaN
1851,4637628408,AK,"1706 Ship Ave, Anchorage, AK",61.222654,-149.851510,NaN
2260,5337318401,AK,"46877 STILLPOINT TRL, HALIBUT COVE, AK",59.592538,-151.231570,NaN
2365,9891427104,AK,"1706 SHIP AVENUE, ANCHORAGE, AK",61.222654,-149.851510,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('02')] #AK
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/11 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

11


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13376/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
356,5048318601,AK,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",64.030600,-145.731320,[022400004001077]
392,2725348408,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,[]
415,5085327008,AK,"1229 Richardson Hwy, DELTA JUNCTION, AK",64.030600,-145.731320,[022400004001077]
1095,4188937110,AK,"525 SECOND STREET, SKAGWAY, AK",40.670782,-73.976908,[]
1601,9864858410,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,[]
1691,5628247103,AK,"6100 A Street, ANCHORAGE, AK",61.165676,-149.882046,[020200025022014]
1847,9635677110,AK,"8840 Old Seward Hwy Ste E, ANCHORAGE, AK",61.140366,-149.865092,[020200027121000]
1851,4637628408,AK,"1706 Ship Ave, Anchorage, AK",61.222654,-149.851510,[020200006013023]
2260,5337318401,AK,"46877 STILLPOINT TRL, HALIBUT COVE, AK",59.592538,-151.231570,[021220012001029]
2365,9891427104,AK,"1706 SHIP AVENUE, ANCHORAGE, AK",61.222654,-149.851510,[020200006013023]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13376/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
356,5048318601,AK,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",64.030600,-145.731320,022400004001077
392,2725348408,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
415,5085327008,AK,"1229 Richardson Hwy, DELTA JUNCTION, AK",64.030600,-145.731320,022400004001077
1095,4188937110,AK,"525 SECOND STREET, SKAGWAY, AK",40.670782,-73.976908,NaN
1601,9864858410,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
1691,5628247103,AK,"6100 A Street, ANCHORAGE, AK",61.165676,-149.882046,020200025022014
1847,9635677110,AK,"8840 Old Seward Hwy Ste E, ANCHORAGE, AK",61.140366,-149.865092,020200027121000
1851,4637628408,AK,"1706 Ship Ave, Anchorage, AK",61.222654,-149.851510,020200006013023
2260,5337318401,AK,"46877 STILLPOINT TRL, HALIBUT COVE, AK",59.592538,-151.231570,021220012001029
2365,9891427104,AK,"1706 SHIP AVENUE, ANCHORAGE, AK",61.222654,-149.851510,020200006013023


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
392,2725348408,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
1095,4188937110,AK,"525 SECOND STREET, SKAGWAY, AK",40.670782,-73.976908,NaN
1601,9864858410,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
356,5048318601,022400004001077
392,2725348408,NaN
415,5085327008,022400004001077
1095,4188937110,NaN
1601,9864858410,NaN
1691,5628247103,020200025022014
1847,9635677110,020200027121000
1851,4637628408,020200006013023
2260,5337318401,021220012001029
2365,9891427104,020200006013023


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/AK_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13,NaN
1,1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13,NaN
2,2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13,NaN
3,3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13,NaN
4,4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13,NaN
...,...,...,...,...,...,...,...,...
2657,2657,7365917001,AK,"188 W NORTHERN LIGHTS BLVD, SUITE 1020, ANCHOR...",61.194692,-149.886020,2.020002e+13,NaN
2658,2658,7400167008,AK,"1119 E. 70TH AVE, ANCHORAGE, AK",61.157184,-149.861692,2.020003e+13,NaN
2659,2659,7483247209,AK,"425 West 58th Ave, anchorage, AK",61.169044,-149.889214,2.020003e+13,NaN
2660,2660,8414228309,AK,"936 E Westpoint Dr, Wasilla, AK",61.582021,-149.427018,2.170001e+13,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13,NaN,20200019003022.0
1,1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13,NaN,21850001001069.0
2,2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13,NaN,20200023011009.0
3,3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13,NaN,20200025012000.0
4,4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13,NaN,21220010002001.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,20200019003022.0
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,21850001001069.0
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,20200023011009.0
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,20200025012000.0
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,21220010002001.0
...,...,...,...,...,...,...
2657,7365917001,AK,"188 W NORTHERN LIGHTS BLVD, SUITE 1020, ANCHOR...",61.194692,-149.886020,20200019001002.0
2658,7400167008,AK,"1119 E. 70TH AVE, ANCHORAGE, AK",61.157184,-149.861692,20200025022018.0
2659,7483247209,AK,"425 West 58th Ave, anchorage, AK",61.169044,-149.889214,20200025012003.0
2660,8414228309,AK,"936 E Westpoint Dr, Wasilla, AK",61.582021,-149.427018,21700008003022.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
392,2725348408,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
1095,4188937110,AK,"525 SECOND STREET, SKAGWAY, AK",40.670782,-73.976908,NaN
1601,9864858410,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN


Convert to File

In [19]:
fips_AK = fips_merge_drop

In [20]:
fips_AK.to_csv("../../../data/state_data/geo/geo_fips/23/AK_fips_scraped.csv") 